In [1]:
import numpy as np
import pandas as pd
import setup
from docplex.mp.model import Model

In [2]:
dfdemand = pd.read_excel('data_or_test.xlsx', sheet_name='Demand')
dfMarkup = pd.read_excel('data_or_test.xlsx', sheet_name='Markup')
dfLastmile = pd.read_excel('data_or_test.xlsx', sheet_name='Last Mile')
dfStock = pd.read_excel('data_or_test.xlsx', sheet_name='Stock')
dfIniStock = pd.read_excel('data_or_test.xlsx', sheet_name='Initial Stock')
dfTransfers = pd.read_excel('data_or_test.xlsx', sheet_name='Transfers')
dfInbound = pd.read_excel('data_or_test.xlsx', sheet_name='Inbound')
dfProduct = pd.read_excel('data_or_test.xlsx', sheet_name='Product')

In [3]:
def removeLetters(df, columnas): # Le saco las letras y le resto 1 al indice para mayor comodidad
    for c in columnas:
        df[c] = pd.to_numeric((df[c].str.replace('\D', '', regex=True))) - 1
    return df

In [4]:
dfMarkup.columns

Index(['Seller', 'Markup (%)'], dtype='object')

In [5]:
dfdemand = removeLetters(dfdemand,['State', 'Seller', 'Product', 'Week'])
dfMarkup = removeLetters(dfMarkup,['Seller'])
dfLastmile = removeLetters(dfLastmile,['FC',"State","Category"])
dfStock = removeLetters(dfStock,['FC',"Seller","Product"])
dfIniStock = removeLetters(dfIniStock,['FC',"Seller","Product"])
dfTransfers = removeLetters(dfTransfers,['FC',"FC.1","Category"])
dfInbound = removeLetters(dfInbound,['FC',"Seller","Category"])
dfProduct = removeLetters(dfProduct,["Product","Category"])

In [6]:
dfdemand

,State,Seller,Product,Week,Demand (unit),Price
0,0,0,0,0,668,1449.19
1,0,0,0,1,887,1983.44
2,0,0,0,2,499,1386.48
3,0,0,0,3,281,889.35
4,0,1,1,0,144,2115.55
...,...,...,...,...,...,...
11995,9,48,298,3,883,294.92
11996,9,49,299,0,137,732.65
11997,9,49,299,1,218,737.64
11998,9,49,299,2,321,2120.03


In [13]:
days = list(range(0,10 ))
fcenters = list(range(0,3 ))
sellers = list(range(0, 50))
zones = list(range(0,10))
products = list(range(0,300))
categorias = list(range(0,5))